In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.rdd.RDD
import org.apache.hadoop.fs.{FileSystem, Path}
import scala.util.matching.Regex
import org.apache.spark.sql.types._

// Initialize SparkSession
val spark = SparkSession.builder()
    .appName("CaseStudy4 - Duplicate Record Removal Pipeline")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.4.0")
    .getOrCreate()


spark = org.apache.spark.sql.SparkSession@3b8a02b3


org.apache.spark.sql.SparkSession@3b8a02b3

In [2]:
val moviesDataPath = "gs://task-dataset-bucket/Day_16_17/movie.csv"
val moviesDF = spark.read.option("header", "true").option("inferSchema", "true").csv(moviesDataPath)

// Show sample data
moviesDF.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

moviesDataPath = gs://task-dataset-bucket/Day_16_17/movie.csv
moviesDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [3]:
val initialCount = moviesDF.count()

// Drop duplicates and count records
val deduplicatedCount = moviesDF.dropDuplicates("movieId", "title").count()

println(s"Initial Count: $initialCount, Deduplicated Count: $deduplicatedCount")

Initial Count: 27278, Deduplicated Count: 27278


initialCount = 27278
deduplicatedCount = 27278


27278

In [4]:
val moviesIn1971 = moviesDF.filter(col("title").contains("(1971)"))
val moviesIn1971Count = moviesIn1971.count()

// Union original DataFrame with the 1971 movies to create duplicates
val unionDF = moviesDF.union(moviesIn1971)
val unionDFCount = unionDF.count()

println(s"Movies in 1971: $moviesIn1971Count, Unioned Count: $unionDFCount")

Movies in 1971: 205, Unioned Count: 27483


moviesIn1971 = [movieId: int, title: string ... 1 more field]
moviesIn1971Count = 205
unionDF = [movieId: int, title: string ... 1 more field]
unionDFCount = 27483


27483

In [5]:
val hdfsPath = "hdfs:///user/day_16_17/duplicate_movies"

// Write unioned data to HDFS
unionDF.write.mode("overwrite").csv(hdfsPath)

hdfsPath = hdfs:///user/day_16_17/duplicate_movies


hdfs:///user/day_16_17/duplicate_movies

In [6]:
val moviesSchema = StructType(Array(
    StructField("movieId", IntegerType, nullable = true),
    StructField("title", StringType, nullable = true),
    StructField("genres", StringType, nullable = true)
))

val moviesUpdatedDF = spark.read.schema(moviesSchema).option("header", "false").csv(hdfsPath)

// Count records in updated DataFrame
val updatedCount = moviesUpdatedDF.count()

println(s"Updated Count: $updatedCount")


Updated Count: 27483


moviesSchema = StructType(StructField(movieId,IntegerType,true),StructField(title,StringType,true),StructField(genres,StringType,true))
moviesUpdatedDF = [movieId: int, title: string ... 1 more field]
updatedCount = 27483


27483

In [7]:
val filteredRDD = moviesUpdatedDF.rdd.map { row =>
    val movieId = row.getAs[Int]("movieId")
    val title = row.getAs[String]("title")
    ((movieId, title), row)
}.reduceByKey((row1, _) => row1)
 .map { case (_, row) => row }

// Count records in filtered RDD
val filteredCount = filteredRDD.count()
println(s"Filtered Count: $filteredCount")


Filtered Count: 27278


filteredRDD = MapPartitionsRDD[65] at map at <console>:40
filteredCount = 27278


27278

In [8]:
// Save filtered data to Avro format
val gsPath = "gs://task-dataset-bucket/Day_16_17/cleaned_movies"

filteredRDD.map { row =>
    val movieId = row.getAs[Int]("movieId")
    val title = row.getAs[String]("title")
    val genres = row.getAs[String]("genres")
    (movieId, title, genres)
}.toDF("movieId", "title", "genres")
.write
.format("avro")
.mode("overwrite")
.save(gsPath)

gsPath = gs://task-dataset-bucket/Day_16_17/cleaned_movies


gs://task-dataset-bucket/Day_16_17/cleaned_movies

In [9]:
val validationDF = spark.read.format("avro").load(gsPath)

// Show sample data and count
validationDF.show()
val finalCount = validationDF.count()

println(s"Final Record Count: $finalCount")

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|  82108|Against the Curre...|               Drama|
|  94133|  Hammer, The (2010)|               Drama|
|  91284|Lonely Passion of...|       Drama|Romance|
|  96717|Pearls of the Cro...|              Comedy|
|  69042|Flash Gordon's Tr...|       Action|Sci-Fi|
|   3530|Smoking/No Smokin...|              Comedy|
|  73449| V.I.P.s, The (1963)|               Drama|
|  27783|Lost Embrace (Abr...|        Comedy|Drama|
|  32369|Panic in the Stre...|Crime|Drama|Film-...|
|   4077|With a Friend Lik...|      Drama|Thriller|
| 127248|  The Auction (2013)|               Drama|
|   1584|      Contact (1997)|        Drama|Sci-Fi|
|  89896|Turin Horse, The ...|               Drama|
|  73392|     Collapse (2009)|         Documentary|
|  85378|Mother Carey's Ch...|       Drama|Romance|
|   5131|How to Kill Your ...|        Comedy|Drama|
|  75421|Gir

validationDF = [movieId: int, title: string ... 1 more field]
finalCount = 27278


27278

In [10]:
spark.stop()